In [241]:
from splinter import Browser
from bs4 import BeautifulSoup as bs

import requests

In [242]:
# Set up Splinter
executable_path = {'executable_path':"C:\Program Files (x86)\msedgedriver.exe"}
browser = Browser('edge', **executable_path)

# Url for scraping
url = "https://www.nba.com/stats/leaders?SeasonType=Regular+Season&Season=2023-24"

browser.visit(url)
html = browser.html

soup = bs(html, 'html.parser')

In [243]:
# Find table element
table = soup.find("table", class_="Crom_table__p1iZz")

# Get table headers
headers = []
for i in range(len(table.thead.find_all('th'))):
    header = table.thead.find_all('th')[i].text
    headers.append(header)

headers

['#',
 'Player',
 'TEAM',
 'GP',
 'MIN',
 'PTS',
 'FGM',
 'FGA',
 'FG%',
 '3PM',
 '3PA',
 '3P%',
 'FTM',
 'FTA',
 'FT%',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'EFF']

In [318]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select

# Set up the Selenium WebDriver (make sure to have the appropriate webdriver installed)
driver = webdriver.Edge(executable_path=r"C:\Program Files (x86)\msedgedriver.exe")

# URL of the webpage containing the dropdown menu
url = "https://www.nba.com/stats/leaders?SeasonType=Regular+Season&Season=2023-24"
driver.get(url)

None


In [319]:
# Find the dropdown element using its HTML attribute, like 'id' or 'name'
dropdown = Select(driver.find_elements_by_class_name("DropDown_select__4pIg9")[5])

# Change the selected option by index, value
print(dropdown.select_by_index(0))

None


In [ ]:
# 

In [244]:
# Scrape data from all pages for current year
# Get the last page of stats for selected year
last_page = int(soup.find_all('select')[-1].contents[-1].text)

data = []
for x in range(last_page):
    # Get data from table
    for row in table.tbody.find_all('tr'):
        cols = row.find_all('td')
        
        if len(cols) == 0:
            cols.find_all('th')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele]) # Get rid of empty values
    browser.find_by_css('.Pagination_button__sqGoH')[1].click()

In [245]:
# data[-1]

In [246]:
# # Get current year of stats
# select_year = soup.find_all("select")[0]
# year_options = []
# for i in range(1, len(select_year)):
#     # Store each year in the years array
#     year_options.append(select_year.contents[i].text)
    
# # Scrape stats for remaining years from dropdown menu
# for x in range(len(year_options)):
#     current_year_stats = year_options[x]
    
#     # Url for scraping the other years
#     url = f"https://www.nba.com/stats/leaders?Season={current_year_stats}"
#     browser.visit(url)
#     html = browser.html
#     soup = bs(html, 'html.parser')
    
    
#     # Find table element
#     table = soup.find("table", class_="Crom_table__p1iZz")

#     # Get data from table
#     data = []
#     for row in table.tbody.find_all('tr'):
#         cols = row.find_all('td')
        
#         if len(cols) == 0:
#             cols.find_all('th')
#         cols = [ele.text.strip() for ele in cols]
#         data.append([ele for ele in cols if ele]) # Get rid of empty values

#         print(f"Gathering data for season: {current_year_stats}")

In [247]:
# import pandas as pd

# # Getting the headers from the data array
# # It is important to remove them from the data array afterwards in order to be parsed correctly by Pandas
# df = pd.DataFrame(data, columns=headers)
# df
